# Financial Forecasting - Demo v3.1

Демонстрация использования класса **FinancialForecaster** для предсказания **доходностей** (returns) на 1-20 дней вперед.

## ⚠️ ВАЖНО: Формат p1-p20

**p1-p20 = предсказанные ДОХОДНОСТИ (returns), НЕ вероятности!**

## Структура решения:
- `solution.py` - основной класс FinancialForecaster (160 моделей)
- `demo.ipynb` - этот ноутбук
- `FORMAT_FIX_CRITICAL.md` - важная информация о формате

## Формат submission:

```csv
ticker,p1,p2,p3,p4,p5,...,p20
AFLT,0.0123,-0.0089,0.0234,0.0012,-0.0156,...,-0.0112
SBER,-0.0067,0.0234,0.0089,0.0012,-0.0145,...,0.0045
```

**Интерпретация**:
- `p1 = 0.0123` → ожидается +1.23% через 1 день
- `p5 = -0.0156` → ожидается -1.56% через 5 дней
- `p20 = 0.0198` → ожидается +1.98% через 20 дней

## 0. Импорты и настройка

In [8]:
import sys
sys.path.append('.')

# Перезагрузка модуля если уже был импортирован
if 'solution' in sys.modules:
    del sys.modules['solution']

from solution import FinancialForecaster
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("✓ Импорты выполнены")
print("✓ Используется обновленная версия solution.py (returns, не probabilities)")

✓ Импорты выполнены
✓ Используется обновленная версия solution.py (returns, не probabilities)


## 1. Инициализация модели

Создаем экземпляр с ансамблем из 160 моделей:
- 2 LGBM (aggressive + conservative) × 2 (regressor + classifier)
- 1 CatBoost × 2 (regressor + classifier)
- 1 Ridge regressor + scaler
- Для каждого из 20 дней = 8 объектов × 20 = 160

In [9]:
model = FinancialForecaster(random_state=42)

print("✓ Модель инициализирована")
print(f"  Конфигураций моделей: {len(model.model_params)}")
print(f"  Модели: {list(model.model_params.keys())}")

✓ Модель инициализирована
  Конфигураций моделей: 7
  Модели: ['lgbm_aggressive', 'lgbm_conservative', 'lgbm_clf_aggressive', 'lgbm_clf_conservative', 'catboost', 'catboost_clf', 'ridge']


## 2. Обучение модели

⏱️ **Время**: ~15-25 минут для всех 160 моделей

Процесс:
1. Загрузка данных
2. Sentiment analysis (keyword-based)
3. Feature engineering (~120 фичей)
4. Кластеризация тикеров
5. Подготовка данных
6. **Обучение 160 моделей** с оптимизацией весов

In [ ]:
train_candles_path = '../forecast_data/candles.csv'
train_news_path = '../forecast_data/news.csv'

# Обучение (долго!)
model.fit(
    candles_path=train_candles_path,
    news_path=train_news_path
)

print("\n" + "="*60)
print("✓ ОБУЧЕНИЕ ЗАВЕРШЕНО")
print("="*60)

ОБУЧЕНИЕ МОДЕЛИ

[1/6] Загрузка данных...
  • Загружено 24197 свечей по 19 тикерам
  • Загружено 25425 новостей

[2/6] Обработка новостей (sentiment analysis)...
  • Средний sentiment: 0.289

[3/6] Feature engineering...
  • Создано 113 фич

[4/6] Кластеризация тикеров...
  • Создано 4 кластеров тикеров

[5/6] Подготовка данных для обучения...
  • Tree features: 75
  • Macro features: 50

[6/6] Обучение ансамбля моделей...
  • Обучение полного ансамбля для p1-p20...
    Модели на горизонт: 2xLGBM + CatBoost + Ridge = 4 модели
    Tree features: 75, Macro features: 50


## 3. Сохранение модели

Сохраняем все 160 моделей + веса ансамблей + scalers в один файл.

💾 Размер: ~50-100 MB

In [ ]:
model.save('trained_model_v3.pkl')

print("✓ Модель сохранена: trained_model_v3.pkl")
print(f"  Моделей сохранено: {len(model.models)}")
print(f"  Scalers: {len(model.scalers)}")
print(f"  Ensemble weights: {len(model.ensemble_weights)}")

## 4. Предсказание на тестовых данных

⏱️ **Время**: ~30 секунд

Генерируем предсказания **доходностей** (returns) для каждого тикера на 20 дней вперед.

In [ ]:
test_candles_path = '../forecast_data/candles_2.csv'
test_news_path = '../forecast_data/news_2.csv'

submission = model.predict(
    candles_path=test_candles_path,
    news_path=test_news_path,
    output_path='submission_v3.csv'
)

print("\n✓ Submission создан: submission_v3.csv")

## 5. Анализ результатов

Проверяем что получили **доходности**, а не вероятности!

In [ ]:
print("="*60)
print("АНАЛИЗ SUBMISSION")
print("="*60)

print(f"\nФорма: {submission.shape}")
print(f"Тикеров: {submission['ticker'].nunique()}")

print("\nПервые 5 строк:")
print(submission.head())

print("\n" + "="*60)
print("СТАТИСТИКА ПО ДОХОДНОСТЯМ")
print("="*60)

returns_cols = [f'p{i}' for i in range(1, 21)]
stats = submission[returns_cols].describe()
print(stats)

print("\n" + "="*60)
print("ПРОВЕРКА ФОРМАТА")
print("="*60)

# Проверяем что это returns, а не вероятности
for col in ['p1', 'p5', 'p10', 'p15', 'p20']:
    min_val = submission[col].min()
    max_val = submission[col].max()
    mean_val = submission[col].mean()
    
    # Returns обычно имеют отрицательные значения или > 1
    # Вероятности всегда в [0, 1]
    is_return = (min_val < 0) or (max_val > 1)
    status = "✅ RETURN" if is_return else "⚠️  PROB?"
    
    print(f"{col}: [{min_val:7.4f}, {max_val:7.4f}], mean={mean_val:7.4f} → {status}")

print("\n" + "="*60)
print("ПРИМЕРЫ ПРЕДСКАЗАНИЙ")
print("="*60)

for ticker in submission['ticker'].head(3):
    row = submission[submission['ticker'] == ticker].iloc[0]
    print(f"\n{ticker}:")
    print(f"  p1  (1 день):  {row['p1']:+.4f} ({row['p1']*100:+.2f}%)")
    print(f"  p5  (5 дней):  {row['p5']:+.4f} ({row['p5']*100:+.2f}%)")
    print(f"  p10 (10 дней): {row['p10']:+.4f} ({row['p10']*100:+.2f}%)")
    print(f"  p20 (20 дней): {row['p20']:+.4f} ({row['p20']*100:+.2f}%)")

## 6. Альтернатива: Загрузка готовой модели

Если модель уже обучена, можно сразу загрузить и делать предсказания.

In [ ]:
# Создаем новый экземпляр
loaded_model = FinancialForecaster()

# Загружаем обученную модель
loaded_model.load('trained_model_v3.pkl')

# Делаем предсказания
submission2 = loaded_model.predict(
    candles_path=test_candles_path,
    news_path=test_news_path,
    output_path='submission_from_loaded.csv'
)

print("✓ Предсказания сделаны с загруженной моделью")
print(f"  Тикеров: {len(submission2)}")
print(f"  Колонок: {len(submission2.columns)}")

## 7. Финальная валидация формата

Убеждаемся что submission в правильном формате.

In [ ]:
def validate_submission(df, name="submission"):
    """Валидация формата submission"""
    print(f"\n{'='*60}")
    print(f"ВАЛИДАЦИЯ: {name}")
    print("="*60)
    
    # Проверка колонок
    required_cols = ['ticker'] + [f'p{i}' for i in range(1, 21)]
    if list(df.columns) == required_cols:
        print("✅ Колонки правильные: ticker,p1,p2,...,p20")
    else:
        print("❌ НЕПРАВИЛЬНЫЕ КОЛОНКИ!")
        return False
    
    # Проверка уникальности тикеров
    if not df['ticker'].duplicated().any():
        print(f"✅ Все тикеры уникальны ({len(df)} строк)")
    else:
        print("❌ ЕСТЬ ДУБЛИКАТЫ ТИКЕРОВ!")
        return False
    
    # Проверка что это returns (не вероятности)
    has_negatives = any(df[f'p{i}'].min() < 0 for i in range(1, 21))
    has_outliers = any(df[f'p{i}'].max() > 1 for i in range(1, 21))
    
    if has_negatives or has_outliers:
        print("✅ Содержит RETURNS (есть отрицательные или >1)")
    else:
        print("⚠️  ВСЕ ЗНАЧЕНИЯ В [0,1] - ВОЗМОЖНО ВЕРОЯТНОСТИ!")
    
    # Статистика
    all_returns = df[[f'p{i}' for i in range(1, 21)]].values.flatten()
    print(f"\nСтатистика всех предсказаний:")
    print(f"  Min: {all_returns.min():.4f}")
    print(f"  Max: {all_returns.max():.4f}")
    print(f"  Mean: {all_returns.mean():.4f}")
    print(f"  Std: {all_returns.std():.4f}")
    
    print("\n✅ ВАЛИДАЦИЯ ПРОЙДЕНА!")
    return True

# Валидируем
validate_submission(submission, "submission_v3.csv")

## 📋 Резюме

### Что мы сделали:
1. ✅ Инициализировали модель (160 моделей)
2. ✅ Обучили на candles.csv + news.csv (~15-25 мин)
3. ✅ Сохранили модель (~50-100 MB)
4. ✅ Сгенерировали submission с **доходностями** (не вероятностями!)
5. ✅ Проверили формат

### Формат submission:
```
ticker,p1,p2,p3,...,p20
AFLT,0.0123,-0.0089,0.0234,...,-0.0112
```

### Архитектура:
- **LGBM Aggressive** (reg + clf) × 20 дней
- **LGBM Conservative** (reg + clf) × 20 дней
- **CatBoost** (reg + clf) × 20 дней
- **Ridge** (reg + scaler) × 20 дней
- **Оптимальные веса** через scipy.optimize

### Следующие шаги:
1. Запустите `test_returns_format.py` для финальной проверки
2. Submit файл `submission_v3.csv`
3. Опционально: оптимизируйте гиперпараметры через `optimize_hyperparams.py`